# Fine-tuning BERT 
Fintech Project

Author: 楊晴雯 

Date: 2022.5.6 

References:

- https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb#scrollTo=X6HrpprwIrIz


In [1]:
from google.colab import drive
drive.mount('/content/drive')
maindir = '/content/drive/MyDrive/FinTech-final-project'
datadir = f'{maindir}/data'
spmdir = f'{maindir}/spm'
modeldir = f'{maindir}/models'
cat_df_path = f'{maindir}/東吳課程_發票資料集/品類資料集/cat_train_v2.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip -q install datasets

In [3]:
from datasets import Dataset, load_metric
import datasets
import joblib, sys
import numpy as np
import pandas as pd

In [4]:
datapath = f'{datadir}/category/category_labels_v2.pkl'
datas = joblib.load(datapath)

In [5]:
cat2idx = joblib.load(f'{datadir}/category/cat2idx.pkl')
idx2cat = {v:k for k, v in cat2idx.items()}

In [6]:
multi = 0 
Rows = []
for idx in range(len(datas)):
  row = []
  data = datas[idx]
  row.append(idx) # first idx
  row.append(data['name'])
  label = list(np.where(data['labels'] >= 1)[0])
  # 
  if len(label) > 1:
    # print(data['name'], label)
    multi += 1
  # else: assert idx2cat[label[0]] == data['product'][0] <- 全對
  row.append(data['labels'])
  row.append(data['seg_name'])
  row.append(data['product'])
  Rows.append(row)
print('How many multi-labeled data?', multi)
print(f'Ratio of multi-labeled data: {multi/len(datas):.4f}')

How many multi-labeled data? 387
Ratio of multi-labeled data: 0.0042


In [7]:
#滿煩的，HF🤗沒有提供multi-label的trainer，要自己寫完

## Dataset 

In [8]:
df = pd.DataFrame(Rows, columns=['id', 'name', 'label', 'seg_name', 'label_name'])

In [9]:
df

,id,name,label,seg_name,label_name
0,0,R視舒坦單支裝人工淚液點眼液0.5ml 24支入,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[R視, 舒坦, 單, 支, 裝, 人工, 淚液, 點眼液, 0.5, ml, , 24,...",[人工淚液]
1,1,優麗舒點眼液15ML,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[優麗, 舒點眼液, 15, ML]",[人工淚液]
2,2,(R)視舒坦單支裝人工,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[(, R, ), 視, 舒坦, 單支, 裝, 人工]",[人工淚液]
3,3,Rohto乾眼淚液,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[Rohto, 乾, 眼, 淚液]",[人工淚液]
4,4,R參天柔潤人工淚液點眼液 5ml 4入,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[R, 參天, 柔潤, 人工, 淚液, 點眼液, 5, ml, 4, 入]",[人工淚液]
...,...,...,...,...,...
92301,92301,mdmmd微醺心機漸層腮紅-玫瑰瑪格麗特,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[mdmmd, 微醺, 心機, 漸層, 腮紅, -, 玫瑰, 瑪格麗特]",[腮紅]
92302,92302,mdmmd微醺心機漸層腮紅-橙酒,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[mdmmd, 微醺, 心機, 漸層, 腮紅, -, 橙酒]",[腮紅]
92303,92303,倫敦芮魅 叛逆玩色腮紅盤005 微醺酒紅,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[倫敦, 芮魅 , 叛逆, 玩色, 腮紅盤, 005 , 微醺, 酒紅]",[腮紅]
92304,92304,PALLADIO柔光幻彩貝殼腮紅微醺蜜桃,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[PALLADIO, 柔光, 幻彩, 貝殼, 腮紅, 微醺, 蜜桃]",[腮紅]


In [10]:
TESTSIZE = 0.2

dataset = Dataset.from_pandas(df)
trainset, testset = dataset.train_test_split(test_size= TESTSIZE).values()
final_dataset = datasets.DatasetDict({"train":trainset,"test":testset})

In [11]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'name', 'label', 'seg_name', 'label_name'],
        num_rows: 73844
    })
    test: Dataset({
        features: ['id', 'name', 'label', 'seg_name', 'label_name'],
        num_rows: 18462
    })
})

## BERT tokenization

In [12]:
!pip -q install transformers

In [13]:
MODELNAME = 'bert-base-chinese'
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(MODELNAME)

In [14]:
def preprocess(examples):
  '''essentailly just tokenization, get input_id, attn_masks, token_type_ids'''
  # print(examples)
  tokenized_name = tokenizer(examples['name'], truncation=True, padding = 'max_length', 
                             max_length = 100)
  examples_ = {k:v for k, v in tokenized_name.items()}
  examples_['name'] = [name for name in examples['name']]
  examples_['label'] = [label for label in examples['label']]
  del examples
  return examples_ 
final_dataset = final_dataset.map(preprocess, batched = True)

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

In [15]:
from transformers import AutoModelForSequenceClassification
N = len(idx2cat)
print(N)
model = AutoModelForSequenceClassification.from_pretrained(MODELNAME, num_labels = N)

217


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'name', 'label', 'seg_name', 'label_name', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 73844
    })
    test: Dataset({
        features: ['id', 'name', 'label', 'seg_name', 'label_name', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18462
    })
})

In [17]:
def collate_fn(batch):
    keys = batch[0].keys()
    print(keys)
    # 'id', 'name', 'label', 'seg_name', 'label_name', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'
    to_tensor = ['input_ids', 'token_type_ids', 'attention_mask']
    new_batch = {k:[d[k] for d in batch] for k in keys if k in to_tensor}
    new_batch = tokenizer.pad(
        new_batch,
        padding= "max_length",
        max_length= 100,
        return_tensors="pt",
    )
    label = [d['label'] for d in batch]
    ids = [d['id'] for d in batch]
    new_batch['label'] = torch.tensor(label, dtype=torch.int64)
    new_batch["id"] = torch.tensor(ids, dtype=torch.int64)
    return new_batch

In [18]:
batch_size = 100
lr = 1e-5
epochs = 20
myseed = 1027 # 43 # 1123
gast = 1
wmst = 20
weight_decay = 0.05

In [19]:
from torch.utils.data import DataLoader
train_loader = DataLoader(final_dataset['train'], batch_size = batch_size, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(final_dataset['test'], batch_size = batch_size, collate_fn=collate_fn, shuffle = False)

In [20]:
split = 'train'
# print(len(final_dataset[split]['id']))
# print(len(final_dataset[split]['label']))
# print(len(final_dataset[split]['seg_name']))
# print(len(final_dataset[split]['input_ids']))
# print(len(final_dataset[split]['attention_mask']))

## Training loop

In [21]:
import pickle
from dataclasses import dataclass
import torch
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [22]:
import random
import torch 
import os
import torch.nn as nn
weight_decay = 0.05
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),
                              weight_decay = weight_decay, 
                              lr = lr)

In [23]:
!pip -q install wandb

In [24]:
import wandb
from datetime import datetime
import os
wandb.login()
now = datetime.now()
runname = now.strftime("%m%d-%H%M")
model_dir = f'{maindir}/neural_network/{runname}'
os.makedirs(model_dir, exist_ok = True)
print('* Name of the run:', runname)

wandb: Currently logged in as: nana2929. Use `wandb login --relogin` to force relogin


* Name of the run: 0506-1151


In [25]:
wandb.init(project = "Fintech Project",
           entity="nana2929", 
           name = runname, 
           config = {
               "batch_size": batch_size,
               "dataset": "麻布數據dataset1.0",
               "learning_rate": lr,
               "epochs": epochs, 
               # "scheduler": SCHETYPE, 
           })

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

colab上gpu記憶體不夠，得挪去server上訓練。

In [28]:
# %%wandb
model.to(device)
for epoch in range(epochs):
  model.train()
  for id, batch in enumerate(train_loader):
      inputs = batch['input_ids'].to(device)
      attnmasks = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      logits = model(inputs, attnmasks).logits
      loss = criterion(logits, labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      print(f'Epoch {epoch+1}| train loss: {loss:.3f}')
    # label length必須要是一致的

dict_keys(['id', 'name', 'label', 'seg_name', 'label_name', 'input_ids', 'token_type_ids', 'attention_mask'])


RuntimeError: ignored

In [ ]:
# 缺compute_accuracy(), logging...